In [22]:

#CUMULANTE DI BINDER

# I dati prodotti dal Metropolis sono correlati, quindi per calcolare l'errore statistico si usa
# il metodo binned bootsrtap :
# 1. Si divide il campione in N/k blocchi di dim k
# 2. Si sceglie un nro random tra 1 e N/k e lo si mette nel nuovo campione. Si ripete finchè il nuovo campione ha dim N
# 3. Si ripete la procedura fino a creare M nuovi campioni, poi si calcola B.C. per ogni campione. 
#    l'errore statistico è calcolato sull'array dei cumulanti 
# L'intera procedura si ripete incrementando k finchè l'errore non si stabilizza.


import numpy as np
import random

def main():
    with open("/Users/monicacesario/Desktop/modulo1/binder_045_200.dat", "r") as f:
        
        xmagn = []
        xene = []
        nlatt = 200

        #  for _ in range(1):
        #     next(f) # Salta le prime tot righe
        for line in f:
            columns = line.split('\t')
            if len(columns) == 4:
                
                xmagn.append(abs(float(columns[2])))
                xene.append(float(columns[3]))
    
    """
    with open("/Users/monicacesario/Desktop/Binder_vs_k.dat", "w") as file1:
        for k in range(1,2000):
            y = bootstrap_error_binder(xmagn,k,200)
            # Scrivo su un file i valori di Binder per poi fare analisi   (GRAFICO DI sigma C.B. IN FUNZIONE DI k)
            file1.write(f"{k}\t{y}\n")  # scrive i valori delle colonne
      
    """
    with open("/Users/monicacesario/Desktop/Binder_045.dat", "a") as file2:
        x = binder_cumulant(xmagn)
        y = bootstrap_error_binder(xmagn,2000,200)
        file2.write(f"{nlatt}\t{x}\t{y}\n")  # scrive i valori delle colonne
        return 0  


#costruisco nuovo campione dall'array originale di dati (k è la dimensione di ogni blocco)
def get_blocks(sample,k):
    #Converto in un array di np e ne prendo la lunghezza (se sample è già un array va bene lo stesso)
    sample=np.asarray(sample)
    N=len(sample)
    print(N)
    # Butto gli ultimi blocchi se k non divide N
    if N%k!=0:
        print(f"Warning: {k} does not divide {N}. Throwing away last {N%k} elements.")
        sample=sample[0:N-N%k]
        N=len(sample)

    #Inizializzo i vettori dei blocchi
    number_blocks=int(N/k)
    blocks=[]

    #Divido il campione in N/k blocchi
    for j in range(number_blocks):
        block=sample[j*k:(j+1)*k]
        blocks.append(block)

    return blocks

#CREA CAMPIONI FAKE
def new_sample(blocks):
    #Prendo la lunghezza e inizializzo il nuovo array
    number_blocks=len(blocks)
    new_sample=[]

    # Scelgo i blocchi a caso per costruire il nuovo campione
    for j in range(number_blocks):
        #Fix one block at random
        u = random.randrange(number_blocks)
        #Put it into new sample
        new_sample.append(blocks[u])

    #Turn list of arrays into one array
    return np.concatenate(new_sample,axis=0)


#BINDER CUMULANT
def binder_cumulant(sample):
    sample=np.asarray(sample)
    sample4=sample**4
    average_sample4=np.average(sample4)
    sample2=sample**2
    average_sample2=np.average(sample2)
    return average_sample4/(average_sample2**2)


#BOOTSTRAP ERROR ON BINDER CUMULANT
def bootstrap_error_binder(sample,k,M):
    #Convert into numpy array and get length
    sample=np.asarray(sample)
    N=len(sample)
    #Divide sample into blocks
    blocks=get_blocks(sample,k)
    #Use blocks to build M new samples
    new_samples=[]
    for i in range(M):
        new_samples.append(new_sample(blocks))
    new_samples=np.asarray(new_samples)
   
    #Compute binder cumulant for each new sample
    cumulant_array=np.zeros(M)
    for j in range(M):
        cumulant_array[j]=binder_cumulant(new_samples[j])
    #Compute variance and statistical error from averages
    bootstrap_variance=np.average(cumulant_array**2)-(np.average(cumulant_array)**2)
    return np.sqrt(bootstrap_variance*M/(M-1))


if __name__ == "__main__":
    main()


100000
